<a href="https://colab.research.google.com/github/Brandon-lz/master/blob/main/aicodes/11%E5%88%9D%E6%AC%A1%E5%AE%9E%E6%88%98/FashionMNIST%E5%AE%9E%E6%88%98_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import  keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

In [ ]:
(x,y), (x_test,y_test) = datasets.fashion_mnist.load_data()
print(x.shape,y.shape,x.max())    #与mnist数据大小相同

(60000, 28, 28) (60000,) 255


In [ ]:
def preprocess(x,y):
  x = tf.cast(x,dtype=tf.float32)/255.
  y = tf.cast(y,dtype=tf.int32)
  return x,y

In [ ]:
batchsz = 128
db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(10000).batch(batchsz)

db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
db_test = db_test.map(preprocess).shuffle(10000).batch(batchsz)

db_iter = iter(db)
sample = next(db_iter)
print(sample[0].shape,sample[1].shape)


(128, 28, 28) (128,)


In [ ]:
#创建神经网络层，共五层
model = Sequential([
    layers.Dense(256,activation=tf.nn.relu),     #[b,784] => [b,256]     初始层
    layers.Dense(128,activation=tf.nn.relu),     #[b,256] => [b,128]     中间层...
    layers.Dense(64,activation=tf.nn.relu),     #[b,128] => [b,64]     
    layers.Dense(32,activation=tf.nn.relu),     #[b,64] => [b,32]           
    layers.Dense(10,activation=tf.nn.relu),     #[b,32] => [b,10]       输出层
                                #最后一层的参数量330 = 32*10 + 10
])

model.build(input_shape=[None,28*28])
model.summary()   #打印网络结构

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_11 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_13 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_14 (Dense)             (None, 10)                330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = optimizers.Adam(lr=1e-3)    #创建优化器

In [ ]:
def main():
  for epoch in range(30):
    for step,(x,y) in enumerate(db):
      x = tf.reshape(x,[-1,28*28])

      with tf.GradientTape() as tape:
        logits = model(x)
        y_onehot = tf.one_hot(y,depth=10)
        loss_mse = tf.reduce_mean(tf.losses.MSE(y_onehot,logits))     #MSE
        loss_ce = tf.losses.categorical_crossentropy(y_onehot,logits,from_logits=True)   #交叉熵
        loss_ce = tf.reduce_mean(loss_ce)
      grads = tape.gradient(loss_ce,model.trainable_variables)      #根据loss求梯度
      optimizer.apply_gradients(zip(grads,model.trainable_variables))    #更新参数，优化过程（w = w - lr*grads）

      if step%100==0 :
        print(epoch,step,'loss=',float(loss_ce))

    #test调试
    total_correct=0
    total_num=0
    for x,y in db_test:
      x = tf.reshape(x,[-1,28*28])
      logits = model(x)
      prob = tf.nn.softmax(logits,axis=1)
      pred = tf.argmax(prob,axis=1)
      pred = tf.cast(pred,dtype=tf.int32)
      correct = tf.equal(pred,y)
      correct = tf.reduce_sum(tf.cast(correct,dtype=tf.int32))

      total_correct += int(correct)
      total_num += x.shape[0]
    acc = total_correct / total_num
    print(epoch,'test acc:',acc)
main()

0 0 loss= 0.36308419704437256
0 100 loss= 0.365620881319046
0 200 loss= 0.25550708174705505
0 300 loss= 0.2903245985507965
0 400 loss= 0.20422053337097168
0 test acc: 0.866
1 0 loss= 0.37439531087875366
1 100 loss= 0.274347722530365
1 200 loss= 0.30140048265457153
1 300 loss= 0.30662065744400024
1 400 loss= 0.26595091819763184
1 test acc: 0.8755
2 0 loss= 0.339439332485199
2 100 loss= 0.28636789321899414
2 200 loss= 0.16893523931503296
2 300 loss= 0.38000428676605225
2 400 loss= 0.3531782627105713
2 test acc: 0.8795
3 0 loss= 0.21474409103393555
3 100 loss= 0.23773255944252014
3 200 loss= 0.21199482679367065
3 300 loss= 0.3250843584537506
3 400 loss= 0.2495397925376892
3 test acc: 0.8784
4 0 loss= 0.3265400230884552
4 100 loss= 0.2805785536766052
4 200 loss= 0.3122848868370056
4 300 loss= 0.1780899465084076
4 400 loss= 0.19995293021202087
4 test acc: 0.8857
5 0 loss= 0.18080806732177734
5 100 loss= 0.2569410800933838
5 200 loss= 0.21574077010154724
5 300 loss= 0.20262539386749268
5 400

可以看到刚开始准确率就能达到0.8以上，对比之前写的三层神经网络，五层神经网络的精度提升是非常明显的